In [2]:
import warnings
import gc
from math import radians,asin
from sklearn.metrics import mean_squared_error,explained_variance_score,mean_absolute_error
from sklearn.model_selection import KFold
import lightgbm as lgb
import seaborn
from tqdm import tqdm_notebook as tqdm
import numpy as np
import math
import os
import time
import json
import joblib
import pandas as pd

warnings.filterwarnings('ignore')

# model

In [3]:
test_B=pd.read_csv('./input/test_B.csv')
sub=test_B[['loadingOrder','timestamp','longitude', 'latitude', 'carrierName','vesselMMSI','onboardDate']]

In [4]:
test_file=pd.read_pickle('./input/test_B_features_7-19.pkl')
test_file=test_file.sort_values(['loadingOrder','timestamp'])
test_file['timestamp']=pd.to_datetime(test_file['timestamp'])
test_file=test_file.groupby(['loadingOrder']).tail(1)
test_file_time=test_file.timestamp
test_file_id=test_file.loadingOrder
test_file=test_file.drop(columns=['loadingOrder','timestamp', 'onboardDate'])

In [5]:
print(test_file.shape[0])
prediction = np.zeros((test_file.shape[0]))

239


In [6]:
model_path='./model/'
for idx in range(5):
    model_name='model_'+str(idx)+'_'+str(0)+'.pkl'
    m=joblib.load(os.path.join(model_path,model_name))
    prediction += m.predict(test_file) / 5

In [7]:
test_file['timestamp']=test_file_time
test_file['loadingOrder']=test_file_id
test_file['predict']=prediction

In [8]:
label_map={i:j for i,j in zip(test_file['loadingOrder'],test_file['predict'])}

In [9]:
test_file['ETA']=(test_file['timestamp'] + test_file['predict'].apply(lambda x:pd.Timedelta(hours=x))).apply(lambda x:x.strftime('%Y/%m/%d  %H:%M:%S'))

In [10]:
ETA_map={i:j for i,j in zip(test_file['loadingOrder'],test_file['ETA'])}

In [11]:
sub['ETA'] = sub['loadingOrder'].map(ETA_map)

sub['creatDate'] = pd.datetime.now().strftime('%Y/%m/%d  %H:%M:%S')

sub.to_csv('./output/sub_final.csv',index=False)